<a href="https://colab.research.google.com/github/ansonmiu0214/C490CW/blob/master/QEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework: Quality Estimation Vectors


In [0]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Importing Data

In [0]:
import os

if not os.path.exists('enzh_data.zip'):
    !wget -O enzh_data.zip https://competitions.codalab.org/my/datasets/download/03e23bd7-8084-4542-997b-6a1ca6dd8a5f
    !unzip enzh_data.zip

TRAIN_EN = 'train.enzh.src'
TRAIN_ZH = 'train.enzh.mt'
TRAIN_SCORES = 'train.enzh.scores'
VAL_EN = 'dev.enzh.src'
VAL_ZH = 'dev.enzh.mt'
VAL_SCORES = 'dev.enzh.scores'
TEST_EN = 'test.enzh.src'
TEST_ZH = 'test.enzh.mt'

## Preprocessing

### English

1. Tokenise with spaCy language model
2. Remove stop words and punctuation
3. Normalise - lemmas

In [13]:
# Downloading spacy models for English

!spacy download en_core_web_md
!spacy link en_core_web_md en300 --force

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_md -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en300
You can now load the model via spacy.load('en300')


In [14]:
# Downloading stop words for English

from nltk import download
from nltk.corpus import stopwords

download('stopwords')
stop_words_en = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Get tokenizer

import spacy

nlp_en = spacy.load('en300')

In [0]:
def preprocess_en(sentence=None, *, keep_stopwords=False):
    def wrapper(sentence):
        text = sentence.lower()
        processed = [token.lemma_ for token in nlp_en.tokenizer(text)]
        processed = [token for token in processed if token.isalpha()]
        if not keep_stopwords:
            processed = [token for token in processed if token not in stop_words_en]
        return processed

    return wrapper if sentence is None else wrapper(sentence)

### Chinese

1. Tokenise with jieba
2. Remove stop words and punctuation

In [0]:
# Download stop words
FILE_STOP_WORDS_ZH = './chinese_stop_words.txt'

if not os.path.exists(FILE_STOP_WORDS_ZH):
    !wget -c https://github.com/Tony607/Chinese_sentiment_analysis/blob/master/data/chinese_stop_words.txt

with open(FILE_STOP_WORDS_ZH, 'r', encoding='utf-8') as f:
    stop_words_zh = [line.rstrip() for line in f]

In [0]:
import jieba

def preprocess_zh(sentence=None, *, keep_stopwords=False):
    def wrapper(sentence):
        tokens = jieba.cut(sentence, cut_all=True)
        processed = [token for token in tokens if token.isalnum()]
        if not keep_stopwords:
            processed = [token for token in processed if token not in stop_words]
        return processed

    return wrapper if sentence is None else wrapper(sentence)

## Language Vocabulary

In [0]:
class Language(object):

    SOS_TOKEN = '<SOS>'
    EOS_TOKEN = '<EOS>'
    UNK_TOKEN = '<UNK>'

    def __init__(self, name):
        self.name = name
        self.word2idx = {}
        self.word2count = {}
        self.idx2word = {0: self.SOS_TOKEN,
                         1: self.EOS_TOKEN,
                         2: self.UNK_TOKEN}
    
    def __len__(self):
        return len(self.idx2word)

    def add_sentence(self, sentence):
        for token in sentence:
            self.add_word(token)

    def add_word(self, word):
        if word not in self.word2idx:
            idx = len(self)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
        
        count = self.word2count.get(word, 0)
        self.word2count[word] = count + 1

    def sent_to_idxs(self, sent):
        return [self.word2idx.get(word, 2) for word in sent]
    
    def __repr__(self):
        return f'Language(name={self.name}) with {len(self)} words'

## Loading Data

In [0]:
# Read from file

with open(TRAIN_EN) as f:
    train_en = f.readlines()
with open(TRAIN_ZH) as f:
    train_zh = f.readlines()
with open(TRAIN_SCORES) as f:
    train_scores = f.readlines()
with open(VAL_EN) as f:
    val_en = f.readlines()
with open(VAL_ZH) as f:
    val_zh = f.readlines()
with open(VAL_SCORES) as f:
    val_scores = f.readlines()
with open(TEST_EN) as f:
    test_en = f.readlines()
with open(TEST_ZH) as f:
    test_zh = f.readlines()

In [63]:
# English data

preprocess_english = preprocess_en(keep_stopwords=True)

train_en_sents = [preprocess_english(sent) for sent in train_en]
val_en_sents = [preprocess_english(sent) for sent in val_en]
test_en_sents = [preprocess_english(sent) for sent in test_en]

EN = Language('EN')
for sent in train_en_sents:
    EN.add_sentence(sent)

print(EN)

print()
print('Sample sentence')
sample_sent_en = train_en_sents[42]
print(sample_sent_en)
print(EN.sent_to_idxs(sample_sent_en))

train_en_idxs = [EN.sent_to_idxs(sent) for sent in train_en_sents]
val_en_idxs = [EN.sent_to_idxs(sent) for sent in val_en_sents]
test_en_idxs = [EN.sent_to_idxs(sent) for sent in test_en_sents]

Language(name=EN) with 19248 words

Sample sentence
['all', 'of', 'the', 'artilleryman', 'record', 'a', 'wound', 'die']
[340, 31, 3, 341, 342, 59, 343, 344]


In [65]:
# Chinese data

preprocess_chinese = preprocess_zh(keep_stopwords=True)

train_zh_sents = [preprocess_chinese(sent) for sent in train_zh]
val_zh_sents = [preprocess_chinese(sent) for sent in val_zh]
test_zh_sents = [preprocess_chinese(sent) for sent in test_zh]

ZH = Language('ZH')
for sent in train_zh_sents:
    ZH.add_sentence(sent)

print(ZH)

print()
print('Sample sentence')
sample_sent_zh = train_zh_sents[42]
print(sample_sent_zh)
print(ZH.sent_to_idxs(sample_sent_zh))

train_zh_idxs = [ZH.sent_to_idxs(sent) for sent in train_zh_sents]
val_zh_idxs = [ZH.sent_to_idxs(sent) for sent in val_zh_sents]
test_zh_idxs = [ZH.sent_to_idxs(sent) for sent in test_zh_sents]

Language(name=ZH) with 23851 words

Sample sentence
['据', '记录', '所有', '6', '名', '炮兵', '都', '受伤', '了']
[483, 484, 485, 267, 486, 487, 488, 489, 17]


In [0]:
# Process scores

def prepare_score(score):
    return float(score)

train_scores = [prepare_score(score) for score in train_scores]
val_scores = [prepare_score(score) for score in val_scores]

In [0]:
# Datasets

train_en_tensors = [torch.LongTensor(sent_idxs) for sent_idxs in train_en_idxs]
train_zh_tensors = [torch.LongTensor(sent_idxs) for sent_idxs in train_zh_idxs]

train_pairs = list(zip(train_en_tensors, train_zh_tensors))
train_set = list(zip(train_pairs, train_scores))
# val_pairs = list(zip(val_en_idxs, val_zh_idxs))
# test_pairs = list(zip(test_en_idxs, test_zh_idxs))

## Models

### RNN Chain

In [0]:
class RNNChain(nn.Module):

    def __init__(self, *, vocab_size, emb_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim

        self.embedding = nn.Embedding(self.vocab_size, self.emb_dim)
        self.rnn = nn.GRU(self.emb_dim, self.emb_dim)

    def forward(self, x, hidden):
        emb = self.embedding(x)
        output = emb.view(1, 1, -1)
        output, hidden = self.rnn(output, hidden)

        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.emb_dim, device=device)

class RegressorLayer(nn.Module):

    def __init__(self, *, emb_dim):
        super().__init__()
        self.emb_dim = emb_dim

        self.hidden = nn.Linear(self.emb_dim, 50)
        self.out = nn.Linear(50, 1)

    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

In [0]:
en_model = RNNChain(vocab_size=len(EN), emb_dim=100)
zh_model = RNNChain(vocab_size=len(ZH), emb_dim=100)
regressor = RegressorLayer(emb_dim=100)

en_opt = torch.optim.SGD(en_model.parameters(), lr=0.01)
zh_opt = torch.optim.SGD(zh_model.parameters(), lr=0.01)
regressor_opt = torch.optim.SGD(regressor.parameters(), lr=0.01)

def RMSELoss(pred, target):
    return -torch.sqrt(torch.mean((pred - target) ** 2))

loss_fn = RMSELoss

In [0]:
def train(en_tensor, zh_tensor, score):
    en_model.zero_grad()
    zh_model.zero_grad()

    en_hidden = en_model.init_hidden()

    for word_idx in en_tensor:
        _, en_hidden = en_model(word_idx, en_hidden)
    
    # print('EN final hidden state', en_hidden)

    zh_hidden = en_hidden
    for word_idx in zh_tensor:
        _, zh_hidden = zh_model(word_idx, zh_hidden)
    
    # print('ZH final hidden state', zh_hidden)

    pred_score = regressor(zh_hidden).squeeze()

    loss = loss_fn(pred_score, score)
    
    # print('Loss', loss)    

    loss.backward()

    regressor_opt.step()
    zh_opt.step()
    en_opt.step()

    return loss.data

In [147]:
def unzip(args):
    return zip(*args)

for eidx in range(100):
    batch = train_set[:100]
    pairs, scores = unzip(batch)
    en_tensors, zh_tensors = unzip(pairs)
    loss = train(en_tensors, zh_tensors, scores)
    print(loss)

RuntimeError: ignored